# Data Science Bootcamp: Introduction to Pandas and Plotly Workshop

This is the jupyter notebook for the Introduction to Pandas and Plotly workshop. In this notebook, we will go over how to use pandas and plotly by running/writing code on an online dataset. At the end, we will have an exercise making visuals together.

## Introduction to Pandas

### Getting Started

To start using pandas like any other package, we need to import it.

In [ ]:
import pandas as pd

(Tip: use the "Run" button to run a cell, or the shortcut command-R, if on a Mac, with the cell highlighted to run it)

Pandas is truly useful if we have data to work with. Once you found the necessary data, the next step is to load the data on jupyter. There are various ways you can load the data depending on where it came from:
* Download the data manually and load it from its location in your file system
* Use the urlretrieve function from the urllib.request to download CSV files from a raw URL
* Use a link to a CSV/JSON/HTML etc. file you would like to use

In this instance, we will simply read a CSV file we have a link to online.

In [ ]:
car_crashes = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/car_crashes.csv')

We can then take a look at the first few rows of our dataframe by issuing the following line of code:

In [ ]:
car_crashes.head()

Now our dataframe is saved in the variable car_crashes and we can use it analyze the data. Let's look at the columns our dataset.

In [ ]:
car_crashes.columns

Notice how this function does not have () at the end. You will notice some functions in pandas do not have parentheses. It's weird and there's probably some complex reason for it that I am not knowledgable on.

Let's store the columns as a list so we can more easily reference it.

In [ ]:
cols = list(car_crashes.columns)
cols

### Data Manipulation

Now that we have our dataset stored as a dataframe, we need to learn how pandas can help us manipulate it so we can get some interesting insights. Run the following line of code below.

In [ ]:
car_crashes[cols].head(7)

Using the [] with a list inside, we were able to indicate what specific columns we want (in this case, all of them), and with the .head(7) function we asked for the first 7 rows of the dataframe. We can also get the last rows with the `tail` function.

In [ ]:
car_crashes[['total', 'speeding']].tail(5)

To get an overall idea of our dataset, we can use the shape function to see the rows and columns in the dataset.

In [ ]:
car_crashes.shape

Additionally we can gather more in-depth information on the dataset, like what are the types of data we have (strings, integers, etc.)

In [ ]:
car_crashes.info()

We observe that most of our columns are of type `float64`, which means they are numbers with a decimal (64 bits) instead of whole numbers (`int`). The `abbrev` column is an object, which basically means it is text.

**Aside:** we can see that we are not dealing with null values--missing information. If any of the rows have null values, we have a few options as to how to deal with them:
* We could simply remove them with the line of code `car_crashes.dropna(subset=['nameOfColHere'], inplace=True)`, where the subset is the list of columns we consider and inplace means to actually change the orignial dataframe, not return a copy
* We can change the values with some estimate, such as the median. For example this function would be `car_crashes['nameOfCol'].fillna(car_crashes['nameOfCol'].median(), inplace=True)` where we fill the na values with the median of all the column and we make sure there is done to the original dataframe (inplace)
* If there are too many null values and making estimations would produce inaccurate data, you can drop the column entirely with the line of code `car_crashes.drop(columns=['nameOfCol'], inplace=True)`
* Sometimes the null values are necessary as they can indicate there is no information for the column (e.g. a `middle name` column for a person who does not possess a middle name). In these instances we may keep the values NA to track which users do not have such information.

If we would like a statistical summary of our numerical data, we can issue the following function

In [ ]:
car_crashes.describe()

To look at subsets of our dataframe that meet a certain criteria (e.g. only car crashes in states that start with the letter A), we can additionally use a list that contains True and False values to subset certain rows we are concerned with. Consider the following below.

In [ ]:
car_crashes[car_crashes['total'] > car_crashes['total'].median()]

We make a True/False list by issuing that we want car crash totals > the median to be true, else false. We implement this list in our square brackets to only return the rows of the dataset that correspond to the True rows of the list.

We can observe that we have all the states with a total number of accidents above the 50th percentile.

Let's sort this in descending order by speeding to see what the highest number of speeding crashes are for this above 50th percentile group of states.

In [ ]:
car_crashes[car_crashes['total'] > car_crashes['total'].median()].sort_values(by=['speeding'], ascending=False).head(3)

Looks like Hawaii has the highest number of speeding related accidents among the >50th percentile.

Let's compare the mean number of alcohol related car crashes for states with total car crashes above and below the 50th percentile. For this we will get a little fancy and use the `groupby` function. First, we need to make an additional column to use for grouping.

In [ ]:
car_crashes['above_fiftieth'] =  car_crashes['total'] > car_crashes['total'].median()
car_crashes.head()

Then we will apply `groupby`.

In [ ]:
car_crashes.groupby('above_fiftieth')['alcohol'].mean()

We can observe that there seems to be more alcohol related accidents in states which have a higher number of car crashes.

Now that we have done that, let's see the abbreviations of the states with the minimum number of total crashes. We can do this in two orders.

In [ ]:
car_crashes[car_crashes['total'] == car_crashes['total'].min()]['abbrev']

In [ ]:
car_crashes['abbrev'][car_crashes['total'] == car_crashes['total'].min()]

**Food for thought:** What is the difference between the first and second line of code?

Now that we have done some basic data manipulation, let's visualize what's happening with our dataset!

## Introduction to Plotly

As a usual first step, let's import the plotly library. However, plotly hosts various different graphing libraries to use, so we will import multiple libraries and save them as acryonyms for ease of use later.

First, let's get a distribution plot of the `total` column. For this we will use the `figure_factory` library. You can read more about these types of plots [here](https://plotly.com/python/figure-factories/).

In [ ]:
import plotly.figure_factory as ff

fig = ff.create_distplot([car_crashes['total']], ['total'])
fig.show()

Plotly is designed to render graphical figures. The rendering process uses the Plotly.js JavaScript library under the hood (although Python developers using this module very rarely need to interact with the Javascript library directly). Figures can be represented in Python either as `dicts` or as instances of the `plotly.graph_objects.Figure` class we can see below. You can read more about the specifics of how `graph_objects` work [here](https://plotly.com/python/creating-and-updating-figures/).

In [ ]:
type(fig)

We can use the fig object to add more elements to our plot, like below.

In [ ]:
# Add title
fig.update_layout(title_text='Distplot of Total Car Crashes')
fig.show()

Let's see if there is any correlation between two of the columns in our dataset. We will be using marginal distribution plots from the `express` library. You can read more about this library [here](https://plotly.com/python/plotly-express/).

In [ ]:
import plotly.express as px

fig = px.scatter(car_crashes, x="total", y="alcohol", marginal_x="histogram", marginal_y="histogram",
                trendline="ols", template="simple_white")
fig.update_layout(title_text='Marginal Distribution of Total Car Crashes and Alcohol related Car Crashes')
fig.show()

We observe that there is a positive correlation between total number of car crashes and total number of alcohol-induced car crashes.

We can also use the parallel coordinates plot to see if there is any trend between the crash types and total number of crashes.

In [ ]:
fig = px.parallel_coordinates(car_crashes[['total', 'speeding', 'alcohol', 'not_distracted', 'no_previous']], 
                              color="total", labels={"total": "Total Crashes",
                              "alcohol": "Alcohol Related", "speeding": "Speed Related",
                              "not_distracted": "Not Distracted", "no_previous": "No Previous",},
                                color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

Higher numbers of total crashes see higher rates of speed related, alcohol related, non-distracted, and first time car crashes. We can also observe that states with smaller total car crashes mainly see non-distracted crashes, indicating to us that distracted drivers induce higher total car crashes.

Finally let's see if there are any regions with a higher number of car crashes than others.

In [ ]:
fig = px.choropleth(car_crashes,
                    locations='abbrev',
                    color='total',
                    color_continuous_scale='spectral_r',
                    hover_name='abbrev',
                    locationmode='USA-states',
                    labels={'total':'Total Car Crashes'},
                    scope='usa')
fig.update_layout(title_text='Total Car Crashes in the US by state')
fig.show()

Southern states seem to have higher totals of car crashes.

## Your Turn

We're going to try some data manipulations on our own now! Get into groups, find a dataset, and try to come up with 2-3 visuals using plotly (bonus tips if you use Pandas to manipulate the data before making the visual)! Feel free to read the tips in the conclusion section.

(Additional Tip: click on the cell block outside of the text box and hit "b" on your keyboard to create a cell **below**, "a" to create a cell **above**, and "d" twice to **delete** the current cell you clicked on)

# Concluding Thoughts

Pandas and Plotly are great tools to get into data science. Pandas is especially useful for data cleaning and manipulation, while Plotly is an easy tool to generate interactive and sophisticated-looking plots. However, there are many more tools out there to use, and we encourage you to use the biggest tool of all for your future data science projects: Googling! A great SWE, DS, etc. is a great googler--they don't memorize lines of codes or libraries, they just know what the right questions are to ask!

Below you will find some links to useful sites for data analysis.

**Datasets:**
* Sample datasets (like car crashes) on [this github repository](https://github.com/mwaskom/seaborn-data)
* [Kaggle](https://www.kaggle.com/)
* [Google's Public Data Search Engine](https://www.google.com/publicdata/directory)
* [US Government's Open Data](data.gov)

**Data Visualization Libraries:**
* [Plotly](https://plotly.com/python/)
* [Matplotlib](https://matplotlib.org/)
* [Seaborn](https://seaborn.pydata.org/)

Additionally, I encourage you to explore more advanced functions in [pandas](https://pandas.pydata.org/). Querying data is a relevant skill for data analysis, and understand Pandas can make it easy to learn R, SQL, and other data analysis-related programming tools!

That's all, happy coding!